In [1]:
import numpy
from numpy.random import randn

from openfermion.transforms._jordan_wigner import *

from openfermion.utils._trotter_exp_to_qgates import *

from openfermion.utils import (commutator, count_qubits, expectation,
                               hermitian_conjugated, normal_ordered,
                               jordan_wigner_sparse, jw_hartree_fock_state,
                               s_squared_operator, sz_operator)

from openfermion.utils._unitary_cc import (uccsd_generator,
                                           uccsd_singlet_generator,
                                           uccsd_singlet_get_packed_amplitudes,
                                           uccsd_singlet_paramsize)

from forestopenfermion.pyquil_connector import qubitop_to_pyquilpauli
from openfermion.ops import QubitOperator

from pyquil.quil import Program
from forestopenfermion import exponentiate
from pyquil.gates import *

from pyquil.api import WavefunctionSimulator
from pyquil import Program

# import to get hamiltonian from psi 
import unittest
import qforte
from openfermion.hamiltonians import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermionpsi4 import run_psi4

import qforte

# Create compact amplitudes list for singlet UCCSD

In [2]:
test_orbitals = 4
test_electrons = 2

# single_amplitudes = randn(*(test_orbitals,) * 2)
# double_amplitudes = randn(*(test_orbitals,) * 4)

sparse_single_amplitudes = numpy.zeros((test_orbitals, test_orbitals))
sparse_double_amplitudes = numpy.zeros((test_orbitals, test_orbitals,
                                        test_orbitals, test_orbitals))

packed_amplitudes = uccsd_singlet_get_packed_amplitudes(
            sparse_single_amplitudes, sparse_double_amplitudes,
            test_orbitals, test_electrons)

print('singles')
print(packed_amplitudes)



singles
[0.0, 0.0]


# Run Psi to generate data for H2

In [3]:
# Set calculation parameters.
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 1
run_fci = 1
bond_length = 0.7414

geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
basis = 'sto-3g'
multiplicity = 1

molecule = MolecularData(
        geometry, basis, multiplicity,
        description=str(round(bond_length, 2)))

# Run Psi4.
molecule = run_psi4(molecule,
                    run_scf=run_scf,
                    run_mp2=run_mp2,
                    run_cisd=run_cisd,
                    run_ccsd=run_ccsd,
                    run_fci=run_fci)

molecule.load()

# Get molecular Hamiltonian.
molecular_hamiltonian = molecule.get_molecular_hamiltonian()

# Get explicit coefficients.
nuclear_repulsion = molecular_hamiltonian.constant
one_body = molecular_hamiltonian.one_body_tensor
two_body = molecular_hamiltonian.two_body_tensor

# Get fermion Hamiltonian.
fermion_hamiltonian = normal_ordered(get_fermion_operator(
    molecular_hamiltonian))

# Get qubit Hamiltonian.
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)

# Test UCCSD for accuracy against FCI using loaded t amplitudes.
ucc_operator = uccsd_generator(
    molecule.ccsd_single_amps,
    molecule.ccsd_double_amps)

ucc_singlet_operator = uccsd_singlet_get_packed_amplitudes(
    molecule.ccsd_single_amps,
    molecule.ccsd_double_amps, test_orbitals, test_electrons)

In [23]:
print(qubit_hamiltonian)

(-0.09886396978427399+0j) [] +
(-0.045322202057777684+0j) [X0 X1 Y2 Y3] +
(0.045322202057777684+0j) [X0 Y1 Y2 X3] +
(0.045322202057777684+0j) [Y0 X1 X2 Y3] +
(-0.045322202057777684+0j) [Y0 Y1 X2 X3] +
(0.1711977489805748+0j) [Z0] +
(0.16862219157249936+0j) [Z0 Z1] +
(0.1205448220329002+0j) [Z0 Z2] +
(0.16586702409067788+0j) [Z0 Z3] +
(0.17119774898057483+0j) [Z1] +
(0.16586702409067788+0j) [Z1 Z2] +
(0.1205448220329002+0j) [Z1 Z3] +
(-0.22278593024287585+0j) [Z2] +
(0.17434844183963868+0j) [Z2 Z3] +
(-0.22278593024287585+0j) [Z3]


In [22]:
print(molecule.ccsd_single_amps)
print(molecule.ccsd_double_amps)


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[[[ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]]

  [[ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]]

  [[ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]]

  [[ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]]]


 [[[ 0.          0.          0.          0.        ]
   [ 0.          0.         

# Print ucc_operator and ucc_compressed_singlet_op

In [4]:
print('ucc_op')
print(ucc_operator)
print('')
print('ucc_singlet_op')
print(ucc_singlet_operator)

ucc_op
0.05677622305 [0^ 2 1^ 3] +
0.05677622305 [1^ 3 0^ 2] +
-0.05677622305 [2^ 0 3^ 1] +
-0.05677622305 [3^ 1 2^ 0]

ucc_singlet_op
[0.0, -0.05677622305]


In [5]:
qubit_op = jordan_wigner(ucc_operator)
# singlet_qubit_op = jordan_wigner(ucc_singlet_operator)
qubit_op *= 1.0j


print('qubit ucc operator')
print(qubit_op)


for qubit_terms, coefficient in qubit_op.terms.items():
    print('')
    print(coefficient)
    for tensor_term in qubit_terms:
        print(tensor_term[1], tensor_term[0])


qubit ucc operator
(-0.0141940557625+0j) [X0 X1 X2 Y3] +
(-0.0141940557625+0j) [X0 X1 Y2 X3] +
(0.0141940557625+0j) [X0 Y1 X2 X3] +
(-0.0141940557625+0j) [X0 Y1 Y2 Y3] +
(0.0141940557625+0j) [Y0 X1 X2 X3] +
(-0.0141940557625+0j) [Y0 X1 Y2 Y3] +
(0.0141940557625+0j) [Y0 Y1 X2 Y3] +
(0.0141940557625+0j) [Y0 Y1 Y2 X3]

(0.0141940557625+0j)
Y 0
Y 1
Y 2
X 3

(-0.0141940557625+0j)
Y 0
X 1
Y 2
Y 3

(-0.0141940557625+0j)
X 0
X 1
Y 2
X 3

(-0.0141940557625+0j)
X 0
Y 1
Y 2
Y 3

(0.0141940557625+0j)
Y 0
X 1
X 2
X 3

(0.0141940557625+0j)
Y 0
Y 1
X 2
Y 3

(0.0141940557625+0j)
X 0
Y 1
X 2
X 3

(-0.0141940557625+0j)
X 0
X 1
X 2
Y 3


In [6]:
# from forestopenfermion.pyquil_connector import qubitop_to_pyquilpauli
# from openfermion.ops import QubitOperator


In [7]:
pauli_sum_representation = qubitop_to_pyquilpauli(qubit_op)
print(pauli_sum_representation)

(0.0141940557625+0j)*Y0*Y1*Y2*X3 + (-0.0141940557625+0j)*Y0*X1*Y2*Y3 + (-0.0141940557625+0j)*X0*X1*Y2*X3 + (-0.0141940557625+0j)*X0*Y1*Y2*Y3 + (0.0141940557625+0j)*Y0*X1*X2*X3 + (0.0141940557625+0j)*Y0*Y1*X2*Y3 + (0.0141940557625+0j)*X0*Y1*X2*X3 + (-0.0141940557625+0j)*X0*X1*X2*Y3


In [8]:
# from pyquil.quil import Program
# from forestopenfermion import exponentiate
# from pyquil.gates import *

pyquil_program = exponentiate(qubit_op)
pyquil_program = Program(X(0), X(1), pyquil_program)
print(pyquil_program)
print(len(pyquil_program))

X 0
X 1
RX(pi/2) 0
RX(pi/2) 1
RX(pi/2) 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RZ(0.028388111525) 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
RX(-pi/2) 0
RX(-pi/2) 1
RX(-pi/2) 2
H 3
RX(pi/2) 0
H 1
RX(pi/2) 2
RX(pi/2) 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RZ(-0.028388111525) 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
RX(-pi/2) 0
H 1
RX(-pi/2) 2
RX(-pi/2) 3
H 0
H 1
RX(pi/2) 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RZ(-0.028388111525) 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
RX(-pi/2) 2
H 3
H 0
RX(pi/2) 1
RX(pi/2) 2
RX(pi/2) 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RZ(-0.028388111525) 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
RX(-pi/2) 1
RX(-pi/2) 2
RX(-pi/2) 3
RX(pi/2) 0
H 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RZ(0.028388111525) 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
RX(-pi/2) 0
H 1
H 2
H 3
RX(pi/2) 0
RX(pi/2) 1
H 2
RX(pi/2) 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RZ(0.028388111525) 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
RX(-pi/2) 0
RX(-pi/2) 1
H 2
RX(-pi/2) 3
H 0
RX(pi/2) 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RZ(0.028388111525) 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
RX(-pi/2) 1
H 2
H 3
H 0
H 1
H 2
RX(

# The correct answer

In [9]:
from pyquil.api import WavefunctionSimulator
from pyquil import Program

print('')
wfn = WavefunctionSimulator().wavefunction(pyquil_program)
print(wfn)


(0.9935598455+0j)|0011> + (-0.1133085762+0j)|1100>


# Now begin importing trial circuit to Qforte

In [10]:
import qforte

In [11]:
#get from openferion
qforte_generator = qforte.build_from_openfermion(qubit_op)
qforte.smart_print(qforte_generator)


 Quantum operator:
(0.0141940557625+0j)[Y0 Y1 Y2 X3]
+ (-0.0141940557625+0j)[Y0 X1 Y2 Y3]
+ (-0.0141940557625+0j)[X0 X1 Y2 X3]
+ (-0.0141940557625+0j)[X0 Y1 Y2 Y3]
+ (0.0141940557625+0j)[Y0 X1 X2 X3]
+ (0.0141940557625+0j)[Y0 Y1 X2 Y3]
+ (0.0141940557625+0j)[X0 Y1 X2 X3]
+ (-0.0141940557625+0j)[X0 X1 X2 Y3]


# Now try to exponentiate the full operator 

In [12]:
expn_generator = qforte.trotterization.trotterize(qforte_generator)
qforte.smart_print(expn_generator)



 Quantum circuit:
[Rx0  Rx1  Rx2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  Rx1  Rx2  H3  Rx0  H1  Rx2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  H1  Rx2  Rx3  H0  H1  Rx2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  H1  Rx2  H3  H0  Rx1  Rx2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx1  Rx2  Rx3  Rx0  H1  H2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  H1  H2  H3  Rx0  Rx1  H2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  Rx1  H2  Rx3  H0  Rx1  H2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx1  H2  H3  H0  H1  H2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  H1  H2  Rx3]


In [13]:
# Set HF state
qc2 = qforte.QuantumComputer(4)
qc2.apply_gate(qforte.make_gate('X',0,0))
qc2.apply_gate(qforte.make_gate('X',1,1))
#qc2.str()

In [14]:
qc2.apply_circuit(expn_generator)
qc2.str()

['(-0.000000 +0.000000 i) |0000>',
 '(0.000000 +0.000000 i) |1000>',
 '(0.000000 -0.000000 i) |0100>',
 '(0.993560 +0.000000 i) |1100>',
 '(0.000000 +0.000000 i) |0010>',
 '(0.000000 +0.000000 i) |1010>',
 '(-0.000000 +0.000000 i) |0110>',
 '(0.000000 +0.000000 i) |1110>',
 '(0.000000 +0.000000 i) |0001>',
 '(-0.000000 +0.000000 i) |1001>',
 '(0.000000 +0.000000 i) |0101>',
 '(0.000000 +0.000000 i) |1101>',
 '(-0.113309 +0.000000 i) |0011>',
 '(0.000000 +0.000000 i) |1011>',
 '(0.000000 +0.000000 i) |0111>',
 '(-0.000000 +0.000000 i) |1111>']

# Now set up UCCSD for variational optemization via VQE

In [15]:
# import to get hamiltonian from psi 
# import unittest
# import qforte
# from openfermion.hamiltonians import MolecularData
# from openfermion.transforms import get_fermion_operator, jordan_wigner
# from openfermionpsi4 import run_psi4
# import numpy

In [16]:
# Set molecule parameters.
# basis = 'sto-3g'
# multiplicity = 1
# bond_length_interval = 0.75
# n_points = 1

# # Set calculation parameters.
# run_scf = 1
# run_mp2 = 0
# run_cisd = 0
# run_ccsd = 1
# run_fci = 1
# delete_input = True
# delete_output = True

# # Generate molecule at different bond lengths.
# hf_energies = []
# fci_energies = []
# bond_lengths = []
# for point in range(1, n_points + 1):
#     bond_length = bond_length_interval * float(point)
#     bond_lengths += [bond_length]
#     geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
#     molecule = MolecularData(
#         geometry, basis, multiplicity,
#         description=str(round(bond_length, 2)))
    
#     # Run Psi4.
#     molecule = run_psi4(molecule,
#                         run_scf=run_scf,
#                         run_mp2=run_mp2,
#                         run_cisd=run_cisd,
#                         run_ccsd=run_ccsd,
#                         run_fci=run_fci)
    
print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
print('Hartree-Fock energy of {} Hartree.'.format(molecule.ccsd_energy))
print('Hartree-Fock energy of {} Hartree.'.format(molecule.fci_energy))

Hartree-Fock energy of -1.1166843870661929 Hartree.
Hartree-Fock energy of -1.137270174684553 Hartree.
Hartree-Fock energy of -1.1372701746571032 Hartree.


In [55]:
active_space_start = 0
active_space_stop = 2

molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))

# Map operator to fermions and qubits.
fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)
#qubit_hamiltonian.compress()
# print('The Jordan-Wigner Molecular Hamiltonian in canonical basis:\n{}'.format(qubit_hamiltonian))

# print('\nBuild Qforte Hamiltonian and generator to calculate the expectation value')
qforte_hamiltonian = qforte.build_from_openfermion(qubit_hamiltonian)
# print('\nThe Molecular Hamiltonian in canonical basis:')


In [73]:
qforte.smart_print(qforte_hamiltonian)

circ_vec = [qforte.QuantumCircuit(), 
qforte.build_circuit('Z_0'), 
qforte.build_circuit('Z_1'), 
qforte.build_circuit('Z_2'), 
qforte.build_circuit('Z_3'), 
qforte.build_circuit('Z_0 Z_1'), 
qforte.build_circuit('Y_0 X_1 X_2 Y_3'),
qforte.build_circuit('Y_0 Y_1 X_2 X_3'),
qforte.build_circuit('X_0 X_1 Y_2 Y_3'),
qforte.build_circuit('X_0 Y_1 Y_2 X_3'),
qforte.build_circuit('Z_0 Z_2'), 
qforte.build_circuit('Z_0 Z_3'), 
qforte.build_circuit('Z_1 Z_2'), 
qforte.build_circuit('Z_1 Z_3'), 
qforte.build_circuit('Z_2 Z_3')] 

coef_vec = [-0.098863969784274,
0.1711977489805748,
0.1711977489805748,
-0.222785930242875,
-0.222785930242875,
0.1686221915724993,
0.0453222020577776,
-0.045322202057777,
-0.045322202057777,
0.0453222020577776,
0.1205448220329002,
0.1658670240906778,
0.1658670240906778,
0.1205448220329002,
0.1743484418396386]

H2_qubit_hamiltonian = qforte.QuantumOperator()
for i in range(len(circ_vec)):
    H2_qubit_hamiltonian.add_term(coef_vec[i], circ_vec[i])
    
qforte.smart_print(H2_qubit_hamiltonian)



 Quantum operator:
(-0.09886396978427399+0j)[]
+ (0.1711977489805748+0j)[Z0]
+ (0.17119774898057483+0j)[Z1]
+ (-0.22278593024287585+0j)[Z2]
+ (-0.22278593024287585+0j)[Z3]
+ (0.16862219157249936+0j)[Z0 Z1]
+ (0.045322202057777684+0j)[Y0 X1 X2 Y3]
+ (-0.045322202057777684+0j)[Y0 Y1 X2 X3]
+ (-0.045322202057777684+0j)[X0 X1 Y2 Y3]
+ (0.045322202057777684+0j)[X0 Y1 Y2 X3]
+ (0.1205448220329002+0j)[Z0 Z2]
+ (0.16586702409067788+0j)[Z0 Z3]
+ (0.16586702409067788+0j)[Z1 Z2]
+ (0.1205448220329002+0j)[Z1 Z3]
+ (0.17434844183963868+0j)[Z2 Z3]

 Quantum operator:
(-0.098863969784274+0j)[]
+ (0.1711977489805748+0j)[Z0]
+ (0.1711977489805748+0j)[Z1]
+ (-0.222785930242875+0j)[Z2]
+ (-0.222785930242875+0j)[Z3]
+ (0.1686221915724993+0j)[Z0 Z1]
+ (0.0453222020577776+0j)[Y0 X1 X2 Y3]
+ (-0.045322202057777+0j)[Y0 Y1 X2 X3]
+ (-0.045322202057777+0j)[X0 X1 Y2 Y3]
+ (0.0453222020577776+0j)[X0 Y1 Y2 X3]
+ (0.1205448220329002+0j)[Z0 Z2]
+ (0.1658670240906778+0j)[Z0 Z3]
+ (0.1658670240906778+0j)[Z1 Z2]
+ (0.

# Set up VQE object

In [18]:
myVQE = qforte.VQE(4, 2, expn_generator, qforte_hamiltonian, 1000) 

In [19]:
circ = qforte.QuantumCircuit()
Exper2 = qforte.Experiment(4, 2, expn_generator, qforte_hamiltonian, 1000)
params2 = [0.0] * 8 
print(Exper2.experimental_avg(params2))

-1.1213978960802022


In [74]:
circ = qforte.QuantumCircuit()
Exper2 = qforte.Experiment(4, 2, circ, qforte_hamiltonian, 500000)
params2 = []
print(Exper2.experimental_avg(params2))

print('')
Exper2 = qforte.Experiment(4, 2, circ, H2_qubit_hamiltonian, 500000)
params2 = []
print(Exper2.experimental_avg(params2))

-1.1167844584883373

-1.1166169476295296


# Perform the porcedure 

In [20]:
optemized = myVQE.do_vqe()
print(optemized.fun)


8
-1.1259301162859798


In [21]:
print(optemized.x)

[ 5.83648682e-05  5.83648682e-05  3.36456299e-05 -2.07366943e-05
  1.67739868e-04  5.83648682e-05 -1.17416382e-04 -4.66918945e-05]
